In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
from bs4 import BeautifulSoup
import requests, csv, string, re
from csv import writer

abc = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
abc = list(abc)
base_url = 'https://www.uktw.co.uk/archive/work/opera/'

In [2]:
def get_info(listings):
    rows = listings.find_all('tr')
    venue, from_date, to_date, company, composer, when_dates, town = ('','','','','','','')
    for row in rows:
        if venue == '':
            try:
                venue = row.find("b", string="Venue").parent.parent.text.replace('Venue', '')
            except:
                None
        if from_date == '':
            try:
                from_date = row.find("b", string="From").parent.parent.text.replace('From', '')
            except:
                None
        if to_date == '':
            try:
                to_date = row.find("b", string="To").parent.parent.text.replace('To', '')
            except:
                None      
        if company == '':
            try:
                company = row.find("b", text=re.compile("Company(.*)")).parent.parent.text.replace('Company', '').strip()
            except:
                None      
            try:
                company = row.find("b", text=re.compile("Producer(.*)")).parent.parent.text.replace('Producer', '').strip()
            except:
                None
            try:
                company = row.find("b", text=re.compile("Presented by(.*)")).parent.parent.text.replace('Presented by', '').strip()
            except:
                None                   
        if composer == '':
            try:
                composer = row.find("b", text=re.compile("Music(.*)")).parent.parent.text.replace('Music', '').strip()                
            except:
                None  
            try:
                composer = row.find("b", text=re.compile("Author(.*)")).parent.parent.text.replace('Author', '').strip()                
            except:
                None       
        if when_dates == '':
            try:
                when_dates = row.find("b", text=re.compile("When(.*)")).parent.parent.text.replace('When', '').strip()                
            except:
                None  
        if town == '':
            try:
                town = row.find("b", text=re.compile("Town(.*)")).parent.parent.text.replace('Town', '').strip()                
            except:
                None                  
    return venue, from_date, to_date, company, composer, when_dates, town

def get_more_info(listings):
    category = 'live'
    details = listings.find('h2', text='Production details')
    if details:
        if len(details.parent.find_all(text = re.compile('Broadcast'))) > 0:
            category = 'broadcast'
    return category

In [3]:
shows= []
for l in abc:
    print(l)
    url = base_url + l
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    codes = soup.find_all('ul')[10].find_all('li')
    for code in codes:        
        opera_name = code.text.strip()
        link = code.find('a').get('href')
        r_show  = requests.get(link)
        listings = BeautifulSoup(r_show.text, "html5lib")
        for i in listings.find_all('i'):
            if len(i.find_all('a')) > 0:
                if i.find('a').get('href') != None:             
                    show = []
                    show.append(opera_name)
                    show.append(i.find('a').get('href'))
                    shows.append(show)

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


In [5]:
listings = []

for idx, show in enumerate(shows):
    if idx%1000==0:
        print(idx)
        
    if show[1] != '':
        try:
            r  = requests.get(show[1])
            soup = BeautifulSoup(r.text, "html5lib")
            work = show[0]
            venue, from_date, to_date, company, composer, when_dates, town  = get_info(soup)
            category = get_more_info(soup)
            listing = []
            listing.append(work)            
            listing.append(venue)
            listing.append(town)
            listing.append(from_date)
            listing.append(to_date)
            listing.append(company)
            listing.append(composer)        
            listing.append(when_dates)         
            listing.append(category)
            listing.append(show[1])            
            listings.append(listing)
        except:
            None

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [6]:
import pandas as pd
headers = ['work', 'venue', 'town', 'from', 'to', 'company', 'composer', 'dates', 'category', 'url']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/raw/uktw.csv')

In [51]:
from datetime import datetime
import re
import math

months_l = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_l = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
        '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 
        '31', '1', '2', '3', '4', '5', '6', '7', '8', '9']

def fix_date(date):
    try:
        date_1 = re.sub('\d+(st|nd|rd|th)', lambda m: m.group()[:-2].zfill(2), date)    
        date_1 = datetime.strptime(date_1, '%d %B %Y').date()
    except:
        print(date)    
    
    return date_1

# def get_num_performances(row):
#     if row['date_from']==row['date_to']:
#         NPERFORM = 1
#     else:
#         days = (row['date_to']-row['date_from']).days
#         if days <= 3:
#             NPERFORM = days
#         elif re.search(r"([01]?[0-9]|2[0-3]):[0-5][0-9]", row['dates']):
#             hours = re.search(r"([01]?[0-9]|2[0-3]):[0-5][0-9]", row['dates']).group(0)
#             if hours==row['dates']:
#                 NPERFORM = days
#             else:
#                 dates = re.sub(r" at ([01]?[0-9]|2[0-3]):[0-5][0-9]", '', row['dates'])
#                 dates = re.split('\.|,| |\*|\n', dates)
#                 days_in_sentence = [x for x in dates if x in days_l]
#                 if len(days_in_sentence) > 0:
#                     NPERFORM = len(days_in_sentence)
#                 elif days <=30:
#                     NPERFORM = min([math.ceil(days/1.5),10])
#                 elif days <=60:
#                     NPERFORM = min([math.ceil(days/4.),15])
#                 elif days >60:
#                     NPERFORM = 15
#         else:
#             if days <=30:
#                 NPERFORM = min([math.ceil(days/1.5),10])
#             elif days <=60:
#                 NPERFORM = min([math.ceil(days/4.),15])
#             elif days >60:
#                 NPERFORM = 15                    
#     return NPERFORM

def get_num_performances(row):
    if row['date_from']==row['date_to']:
        NPERFORM = 1
    else:
        days = (row['date_to']-row['date_from']).days + 1
        if days <= 3:
            NPERFORM = days
        elif re.search(r"([01]?[0-9]|2[0-3]):[0-5][0-9]", row['dates']):
            hours = re.search(r"([01]?[0-9]|2[0-3]):[0-5][0-9]", row['dates']).group(0)
            dates = re.sub(r" at ([01]?[0-9]|2[0-3]):[0-5][0-9]", '', row['dates'])
            dates = re.split('\.|,| |\*|\n', dates)
            days_in_sentence = [x for x in dates if x in days_l]
            if len(days_in_sentence) > 0:
                NPERFORM = len(days_in_sentence)
            elif days <=30:
                NPERFORM = min([math.ceil(days/1.5),10])
            elif days <=60:
                NPERFORM = min([math.ceil(days/4.),15])
            elif days >60:
                NPERFORM = 15
        else:
            if days <=30:
                NPERFORM = min([math.ceil(days/1.5),10])
            elif days <=60:
                NPERFORM = min([math.ceil(days/4.),15])
            elif days >60:
                NPERFORM = 15
    return NPERFORM

In [53]:
#Load and finish the df.
import pandas as pd
df = pd.read_csv('../../data/raw/uktw.csv', index_col=0)
df['from'].fillna(df['to'], inplace=True)
df['date_from'] = df.apply(lambda row: fix_date(row['from']), axis=1)
df['date_to'] = df.apply(lambda row: fix_date(row['to']), axis=1)
df['dates'].fillna('', inplace=True)
df['num_performances'] = df.apply(lambda row: get_num_performances(row), axis=1)

In [54]:
output_column_names = ['work', 'composer', 'venue', 'town', 'company', 'date_from', 'date_to', 'num_performances', 'url']
df = df[output_column_names]
df.to_csv('../../data/processed/listings/uk_theatreweb.csv')